# 1. import necessaries library for setting up the environment

In [1]:
#main library for data analysis
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#library for json file
import json 
#convert address to latitude, longitude values 
from geopy.geocoders import Nominatim
#library for handle request
import requests
#transform json file into pandas df
from pandas.io.json import json_normalize
#matplotlib for plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors
#slkearn for KMeans Cluster
from sklearn.cluster import KMeans
#folium for mapping 
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

# 2. Web scrapping 

In [2]:
#import Beautifulsoup for scrapping wiki
from bs4 import BeautifulSoup

In [3]:
#store wiki page in url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#open url with beautiful soup
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
#print(soup)

#### if you print soup you get text in html so we needed to scrape them into table with 3 columns
#### Create a [ ] so we can stored data within 3 cloumns 

In [4]:
#create an empty list to stored 
postalCode_list = [] 
borough_list = [] 
neighborhood_list = []
#create for loop to find tag<tr> and tag<td>
for i in soup.find('table').find_all('tr'):
    x = i.find_all('td')
    if (len(x) > 0):
        postalCode_list.append(x[0].text)
        borough_list.append(x[1].text)
        neighborhood_list.append(x[2].text.rstrip('\n'))
print( 'This is postalCode_list',postalCode_list[0:5])
print('This is borough_list',borough_list[0:5])
print('This is neihborhood_list',neighborhood_list[0:5])

This is postalCode_list ['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
This is borough_list ['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
This is neihborhood_list ['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront']


#### Let's put these in DataFrame

In [5]:
df = [('PostalCode', postalCode_list), ('Borough', borough_list),('Neighborhood', 
       neighborhood_list)]

In [6]:
df1 = pd.DataFrame.from_dict(dict(df))
df1.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### As we can see there are 'Not assigned' in Borough, Neighborhood columns, we don't want to deal with missing valued

In [8]:
df1_dropna = df1[df1['Borough'] != 'Not assigned'].reset_index(drop = True)
df1_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### From the instruction,
#### 'More than one neighborhood can exist in one postal code area. 
#### For example, in the table on the Wikipedia page, 
#### you will notice that M5A is listed twice and has two neighborhoods: 
#### Harbourfront and Regent Park. These two rows will be combined 
#### into one row with the neighborhoods separated with a comma 
#### as shown in row 11 in the above table
##### let's grouped neighbood by Postal and Borough

In [9]:
df1_dropna_grouped = df1_dropna.groupby(['PostalCode','Borough'],
as_index = False).agg(lambda x: ','.join(x))
df_toronto = df1_dropna_grouped
df_toronto.loc[80:85]

,PostalCode,Borough,Neighborhood
80,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
81,M6N,York,"The Junction North,Runnymede"
82,M6P,West Toronto,"High Park,The Junction South"
83,M6R,West Toronto,"Parkdale,Roncesvalles"
84,M6S,West Toronto,"Runnymede,Swansea"
85,M7A,Queen's Park,Not assigned


#### According to the instruction,
#### 'If a cell has a borough but a Not assigned neighborhood, 
#### then the neighborhood will be the same as the borough. 
#### So for the 9th cell in the table on the Wikipedia page, 
#### the value of the Borough and the Neighborhood columns will be Queen's Park.'
#### 85th has 'Not assigned' valued in Neighborhood, so we decied to assign <br><br> the value to be the same as Borough

In [10]:
neighborhood_na = df1_dropna_grouped['Neighborhood'] == 'Not assigned'
#located the 'Not assigned' and assign to the same value as Borough
df1_dropna_grouped.loc[neighborhood_na, 'Neighborhood'] = df1_dropna_grouped.loc[neighborhood_na, 'Borough']
df1_dropna_grouped.loc[80:85]

,PostalCode,Borough,Neighborhood
80,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
81,M6N,York,"The Junction North,Runnymede"
82,M6P,West Toronto,"High Park,The Junction South"
83,M6R,West Toronto,"Parkdale,Roncesvalles"
84,M6S,West Toronto,"Runnymede,Swansea"
85,M7A,Queen's Park,Queen's Park


In [11]:
df = df1_dropna_grouped
df.shape

(103, 3)

# 3. Getting Coordinates from csv file

#### After we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [12]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('download completed!')

download completed!


In [13]:
coor = pd.read_csv('Geospatial_data.csv')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### As you can see in coor DataFrame has the same column as our df_toronto DataFrame so let's merge them together to get a full df 

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
#First we indexes by df.PostalCode = coor.Postal Code and inner join(intersect)
df1 = df.set_index('PostalCode')
coor1 = coor.set_index('Postal Code')
df_toronto_full = pd.concat([df1, coor1], axis = 1, join = 'inner')
df_toronto_full.reset_index(inplace = True)
df_toronto_full.rename(columns = {'index':'PostalCode'}, inplace = True)
Toronto_df = df_toronto_full

In [16]:
Toronto_df.loc[1:11]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_df['Borough'].unique()),
        Toronto_df.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


# 4. Explore and Cluster

In [19]:
#Use geopy library to get the latitude and longitude values of Toronto.
#In order to define an instance of the geocoder, we need to define a user_agent. 
#We will name our agent toronto_neigh-agent, as shown below.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_neigh-agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [23]:
# Create a Visualize toronto_map by using lat, long
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map 
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

##### According to the instruction, Explore and cluster the neighborhoods in Toronto. 
##### 'You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.'

In [31]:
# As you can see on our Toronto_df we have 11 bourough and 103 neighborhoods but we can use only borough that have 'Toronto' in it for the sake of simplisity
print('dataframe borough from Toronto_df has {} borough and {} neighborhoods'.format(len(Toronto_df['Borough'].unique()), Toronto_df.shape[0]))

dataframe borough from Toronto_df has 11 borough and 103 neighborhoods


In [32]:
# from df_Toronto columns Borough get str that contain 'Toronto' and assign to df_toronto
df_toronto = Toronto_df[Toronto_df['Borough'].str.contains('Toronto')]
# After that we need to reset the index for our new variable 
df_toronto.reset_index(drop = True, inplace = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
#Feature size of feature set and shape
print('dataframe has {} boroughs and {} neighborhoods.'.format(len(df_toronto['Borough'].unique()), df_toronto.shape[0]))
print('The shape of dataframe is:', df_toronto.shape)

dataframe has 4 boroughs and 38 neighborhoods.
The shape of dataframe is: (38, 5)


In [43]:
# Create a Visualize toronto_map by using lat, long but this time use df_toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

# 4. Foursquare api 

In [47]:
CLIENT_ID = 'VFQIALTA3BOXJMGBWZ1JAHCTOE5W1FOJ04V1Y54JS0E5E5TD'  #Foursquare ID
CLIENT_SECRET = 'XNYUP24ROZRLUKQRZRURJWPUOZQV3HXV3FKO2QAFW2UZYDU1' #Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Connecting....')

Your credentails:
CLIENT_ID: VFQIALTA3BOXJMGBWZ1JAHCTOE5W1FOJ04V1Y54JS0E5E5TD
CLIENT_SECRET:XNYUP24ROZRLUKQRZRURJWPUOZQV3HXV3FKO2QAFW2UZYDU1
Connecting....


In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        #API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
# use function getNearbyVenues to acquire names, lat, long from df_toronto and assign to Toronto_venues
Toronto_venues = getNearbyVenues(names = df_toronto['Neighborhood'],
                                   latitudes= df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [53]:
Toronto_venues.head()
# group by Neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


# 5. Analyze each Neighborhood

In [86]:
# one hot encoding
Toronto_endcode = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#add neighborhood cols
Toronto_endcode['Neighborhood'] = Toronto_venues['Neighborhood']
Toronto_endcode.shape

(1704, 241)

In [79]:
# After add Neighborhood col, let's move it to the front
# Convert Toronto_endcode cols to list and assign to cols_list 
cols_list = Toronto_endcode.columns.tolist()
# move Neighborhood to index 0 of cols_list
cols_list.insert(0, cols_list.pop(cols_list.index('Neighborhood')))
cols_list
# Reindex cols in Toronto_endcode by assign to cols_list and assign it back to Toronto_endcode
Toronto_endcode = Toronto_endcode.reindex(columns = cols_list)
Toronto_endcode.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [84]:
# After that groub Toronto_endcode by average of Neighborhood cols for weights, reset_index and round it to 2 decimal palces
Toronto_grouped = Toronto_endcode.groupby('Neighborhood').mean().reset_index().round(2)
Toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.0,0.0,0.01,0.01,0.0,0.03,0.0,0.00,0.0,0.0,0.00,0.02,0.0,0.04,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.01,0.00,0.0,0.01,0.03,0.00,0.0,0.01,0.03,0.01,0.0,0.00,0.05,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.07,0.0,0.0,0.0,0.01,0.00,0.00,0.02,0.00,0.03,0.0,0.0,0.00,0.0,0.0,0.01,0.01,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.02,0.0,0.0,0.01,0.0,0.0,0.01,0.00,0.01,

In [89]:
# After grouped the row has decrease from 1704 to 38
Toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.0,0.0,0.01,0.01,0.0,0.03,0.0,0.00,0.0,0.0,0.00,0.02,0.0,0.04,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.01,0.00,0.0,0.01,0.03,0.00,0.0,0.01,0.03,0.01,0.0,0.00,0.05,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.07,0.0,0.0,0.0,0.01,0.00,0.00,0.02,0.00,0.03,0.0,0.0,0.00,0.0,0.0,0.01,0.01,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.02,0.0,0.0,0.01,0.0,0.0,0.01,0.00,0.01,

In [96]:
# Assign 5 to num_to_venues 
num_top_venues = 5
# Create function that create temp that contain venue and freq of each hood in neighborhood 
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----") #print name of the neighborhood
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index() #reset index
    temp.columns = ['venue','freq'] # create 2 temp col (venue, freq)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.04
4      Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Garden Center  0.06
3                Park  0.06
4          Comic Shop  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Term

In [111]:
# Create a function that return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [121]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Hotel,Restaurant,Breakfast Spot,Burger Joint,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Jazz Club
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Sandwich Place,Restaurant,Caribbean Restaurant,Climbing Gym,Grocery Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Coffee Shop,Sculpture Garden,Harbor / Marina,Bar,Airport Gate


#### Cluster by venues

In [128]:
# k = 5 
k = 5
# Drop col Neighborhood and assign to cluster
cluster = Toronto_grouped.drop('Neighborhood', axis = 1)
# k-means Clustering
clustering = KMeans(n_clusters = k, random_state = 1).fit(cluster)
#check cluster label
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [144]:
# add label
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# Assign df_toronto to merged_group cause we want to join neighborhoods_venues_sorted and df_toronto
merged_group = df_toronto
#join
merged_group = merged_group.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on= 'Neighborhood')
merged_group.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Pizza Place,Pet Store,Movie Theater,Sushi Restaurant,Steakhouse,Liquor Store,Ice Cream Shop,Italian Restaurant,Pub,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Fish Market,Coworking Space,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Dim Sum Restaurant,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [146]:
merged_group['Cluster Labels'].value_counts()

1    33
0     2
4     1
3     1
2     1
Name: Cluster Labels, dtype: int64

In [148]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_group['Latitude'], merged_group['Longitude'], merged_group['Neighborhood'], merged_group['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters